In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

plt.style.use('tableau-colorblind10')

Clocks move in hr. Every cycle the program will update each of the stocks using the rules/laws established below

In [2]:
# General simulation parameters
Nt = 24*365 # hours
Nt

8760

In [3]:
# Law: make it rain for 50% of the simulated hours at random cumulative intervals
#      Also, randomly apply dry regions to the precipitation map

precipitation = [0]*Nt
for i in range(int(Nt/4)):
    precipitation[np.random.randint(Nt-1)] += np.random.randint(2) # this command chooses a random assigns random position in list (precipitation[ ]) value 0 or 1 (can be changed by changing the 2)

for n in range(200): #200 days will be dry with 
    random_index = np.random.randint(Nt-1-24) #values counting from zero (-1) remove a day (-24)
    for i in range(random_index, random_index+24): # make the random value +the next 24 ones all zero
        precipitation[i] = 0

In [4]:
# Reservoirs
fracture = [None]*Nt
soil     = [None]*Nt

# Water sink(s)
plant = [None]*Nt

# Initial conditions
fracture[0] = 10 # liters #this shall be determined in function of the data 
soil[0]     = 10
plant[0]    = 1

In [5]:
# Record all flow channels
q1 = [None]*Nt # precipitation -> fracture /not likely 
q2 = [None]*Nt # precipitation -> soil
q3 = [None]*Nt # soil -> fracture
q4 = [None]*Nt # fracture -> plant
q5 = [None]*Nt # soil -> plant
q6 = [None]*Nt # plant -> air
sapflow = [None]*Nt


In [17]:
for n in range(Nt):
    # Precipitation
    # Law: 50% to each reservoir
    if precipitation[n] > 0:
        q1[n] = precipitation[n]*0.5 #here I need to specify how much of the precipitation stays in soil and how much goes to the fracture
        q2[n] = precipitation[n]*0.5
        fracture[n] += q1[n]
        soil[n] += q2[n]
    else:
        q1[n] = 0
        q2[n] = 0


In [16]:
for n in range(Nt):
    # Precipitation
    # Law: 50% to each reservoir
    if precipitation[n] > 0:
        q1[n] = precipitation[n]*0.5 #here I need to specify how much of the precipitation stays in soil and how much goes to the fracture
        q2[n] = precipitation[n]*0.5
        fracture[n] += q1[n]
        soil[n] += q2[n]
    else:
        q1[n] = 0
        q2[n] = 0

    # Soil-fracture mechanism
    # Law: water flows from soil to fracture
    #      flow rate is proportional to reservoir volume
    if soil[n] > 0:
        q3[n] = 0.0001*soil[n]
        fracture[n] += q3[n]
        soil[n] -= q3[n]
    else:
        q3[n] = 0

    # Fracture-plant mechanism
    # Law: same as before
    if fracture[n] > 0:
        q4[n] = 0.001*fracture[n]
        plant[n] += q4[n]
        fracture[n] -= q4[n]
    else:
        q4[n] = 0

    # Soil-plant mechanism
    # Law: same as before
    if soil[n] > 0:
        q5[n] = 0.001*fracture[n]
        plant[n] += q5[n]
        soil[n] -= q5[n]
    else:
        q5[n] = 0

    # Plant sap-flow and transpiration
    # Law: transpiration = q6
    sapflow[n] = q4[n] + q5[n]
    if plant[n] > 0:
        q6[n] = sapflow[n]
        plant[n] -= q6[n]
        if sapflow[n] == 0:
            print(sapflow[n])
    else:
        q6[n] = 0

    # Copy computed results to future step
    if n < (Nt-1):
        soil[n+1] = soil[n]
        fracture[n+1] = fracture[n]
        plant[n+1] = plant[n]

In [ ]:
#This is for my personal record, line (1) is equivalent to lines below (2) and it meants to do a random precipitation value every 24 h 
# n % 24 remain of X number of groups of 24 I can make from n, if n<24 I cannot make a single group out of 24 so the original value remains 
# for n>24 , e.g 25 I can group 24 and I have 1 extra, for 75 I can make 3 groups of 24 and I have 3 extra as residual 
# with that technique I make my original set 0,1,2,3,...8759 to become 0,1,2,...23,0,1,2,...,23,0,1,2....
#then we set the condition that exactly at midnight every day it rains 

# (1) # precipitation = [np.random.randint(10) if n % 24 == 0 else 0 for n in range(Nt)]

# (2) # 
#P=[]
#for n in range (Nt): 
#    if n % 24 == 0: 
#        precipitation=np.random.randint(10)
#        P.append(precipitation)
#    else:
#        precipitation=0
#        P.append(precipitation)